![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/work-with-data/dataprep/how-to-guides/append-columns-and-rows.png)

# Append Columns and Rows

Often the data we want does not come in a single dataset: they are coming from different locations, have features that are separated, or are simply not homogeneous. Unsurprisingly, we typically want to work with a single dataset at a time.

Azure ML Data Prep allows the concatenation of two or more dataflows by means of column and row appends.

We will demonstrate this by defining a single dataflow that will pull data from multiple datasets.

## Table of Contents
[append_columns(dataflows)](#append_columns)<br>
[append_rows(dataflows)](#append_rows)

<a id="append_columns"></a>

## `append_columns(dataflows)`
We can append data width-wise, which will change some or all existing rows and potentially adding rows (based on an assumption that data in two datasets are aligned on row number).

However we cannot do this if the reference dataflows have clashing schema with the target dataflow. Observe:

In [ ]:
from azureml.dataprep import auto_read_file

In [ ]:
dflow = auto_read_file(path='../data/crime-dirty.csv')
dflow.head(5)

In [ ]:
dflow_chicago = auto_read_file(path='../data/chicago-aldermen-2015.csv')
dflow_chicago.head(5)

In [ ]:
from azureml.dataprep import ExecutionError
try:
    dflow_combined_by_column = dflow.append_columns([dflow_chicago])
    dflow_combined_by_column.head(5)
except ExecutionError:
    print('Cannot append_columns with schema clash!')

As expected, we cannot call `append_columns` with target dataflows that have clashing schema.

We can make the call once we rename or drop the offending columns. In more complex scenarios, we could opt to skip or filter to make rows align before appending columns. Here we will choose to simply drop the clashing column.

In [ ]:
dflow_combined_by_column = dflow.append_columns([dflow_chicago.drop_columns(['Ward'])])
dflow_combined_by_column.head(5)

Notice that the resultant schema has more columns in the first N records (N being the number of records in `dataflow` and the extra columns being the width of the schema of our reference dataflow, chicago, minus the `Ward` column). From the N+1th record onwards, we will only have a schema width matching that of the `Ward`-less chicago set.

Why is this? As much as possible, the data from the reference dataflow(s) will be attached to existing rows in the target dataflow. If there are not enough rows in the target dataflow to attach to, we simply append them as new rows.

Note that these are appends, not joins (for joins please reference [Join](join.ipynb)), so the append may not be logically correct, but will take effect as long as there are no schema clashes.

In [ ]:
# Ward-less data after we skip the first N rows
dflow_len = dflow.row_count
dflow_combined_by_column.skip(dflow_len).head(5)

<a id="append_rows"></a>

## `append_rows(dataflows)`
We can append data length-wise, which will only have the effect of adding new rows. No existing data will be changed.

In [ ]:
from azureml.dataprep import auto_read_file

In [ ]:
dflow = auto_read_file(path='../data/crime-dirty.csv')
dflow.head(5)

In [ ]:
dflow_spring = auto_read_file(path='../data/crime-spring.csv')
dflow_spring.head(5)

In [ ]:
dflow_chicago = auto_read_file(path='../data/chicago-aldermen-2015.csv')
dflow_chicago.head(5)

In [ ]:
dflow_combined_by_row = dflow.append_rows([dflow_chicago, dflow_spring])
dflow_combined_by_row.head(5)

Notice that neither schema nor data has changed for the target dataflow.

If we skip ahead, we will see our target dataflows' data.

In [ ]:
# chicago data
dflow_len = dflow.row_count
dflow_combined_by_row.skip(dflow_len).head(5)

In [ ]:
# crimes spring data
dflow_chicago_len = dflow_chicago.row_count
dflow_combined_by_row.skip(dflow_len + dflow_chicago_len).head(5)